In [1]:
import requests
from bs4 import BeautifulSoup
header = {'user-agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/72.0.3626.119 Safari/537.36'}

In [2]:
def getDownload(url, param = None, retries = 3):
    resp = None
    try:
        resp = requests.get(url, params = param, headers = header)
        resp.raise_for_status()
    except requests.exceptions.HTTPError as e:
        if 500 <= resp.status_code < 600 and retries > 0:
            print('Retries : {0}'.format(retries))
            return getDownload(url, param, retries -1)
        else:
            print(resp.status_code)
            print(resp.reason)
            print(resp.request.headers)
            
    return resp

In [3]:
url = 'http://example.webscraping.com/places/ajax/search.json'
param = {
    'search_term':'korea',
    'page_size':10,
    'page':0
} 
html = getDownload(url,param)
html.headers

{'Server': 'nginx', 'Date': 'Wed, 25 Sep 2019 06:19:50 GMT', 'Content-Type': 'application/json', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Vary': 'Accept-Encoding', 'X-Powered-By': 'web2py', 'Set-Cookie': 'session_id_places=True; httponly; Path=/', 'Expires': 'Wed, 25 Sep 2019 06:19:50 GMT', 'Pragma': 'no-cache', 'Cache-Control': 'no-store, no-cache, must-revalidate, post-check=0, pre-check=0', 'Content-Encoding': 'gzip'}

In [4]:
for _ in html.json()['records']:
    print(_['pretty_link'])
    dom = BeautifulSoup(_['pretty_link'], 'lxml')
    print([_['href'] for _ in dom.find_all('a')])
    
    break

<div><a href="/places/default/view/North-Korea-165"><img src="/places/static/images/flags/kp.png" /> North Korea</a></div>
['/places/default/view/North-Korea-165']


In [5]:
import re

text = html.json()['records'][0]['pretty_link']
group = re.findall("<a href=\"(.+)\"><img src=\"(.+)\" /> (.+)</a>",text)
requests.compat.urljoin(url, group[0][0]), \
requests.compat.urljoin(url, group[0][1], group[0][2])

('http://example.webscraping.com/places/default/view/North-Korea-165',
 'http://example.webscraping.com/places/static/images/flags/kp.png')

## Selenium

In [9]:
!pip install selenium
from selenium import webdriver


In [14]:
path = r'C:\Users\user\Downloads\chromedriver.exe'
browser = webdriver.Chrome(path)

In [15]:
url = 'http://example.webscraping.com/places/default/search'
browser.get(url)

## id

In [16]:
browser.find_element_by_id('search_term').clear()
browser.find_element_by_id('search_term').send_keys('korea')
browser.find_element_by_id('search').click()

## css_selector

In [17]:
browser.find_element_by_css_selector('#search_term').clear()
browser.find_element_by_css_selector('#search_term').send_keys('korea')
browser.find_element_by_css_selector('#search').click()

## xpath

- xpath => xml tree 탐색

- . => 현재위치

- // => 자손
- / => 자식
- [@어트리뷰트 = "값"]
- id = results
- [@id = 'results']

In [19]:
browser.find_element_by_xpath('//input[@id="search_term"]').clear()
browser.find_element_by_xpath('//input[@id="search_term"]').send_keys('korea')
browser.find_element_by_xpath('//input[@id="search"]').click()

In [20]:
for tag in browser.find_elements_by_xpath('//div[@id="results"]//a'):
    print(tag.text)
    print(tag.get_attribute('href'))

North Korea
http://example.webscraping.com/places/default/view/North-Korea-165
South Korea
http://example.webscraping.com/places/default/view/South-Korea-211


## BeautifulSoup

In [21]:
dom = BeautifulSoup(browser.page_source, 'lxml')
for _ in dom.select_one('#results').select('img'):
    print(requests.compat.urljoin(browser.current_url, _['src']))
    
print('')    
    
for _ in dom.select_one('#results').select('a'):
    print(requests.compat.urljoin(browser.current_url, _['href']))

http://example.webscraping.com/places/static/images/flags/kp.png
http://example.webscraping.com/places/static/images/flags/kr.png

http://example.webscraping.com/places/default/view/North-Korea-165
http://example.webscraping.com/places/default/view/South-Korea-211


In [22]:
browser.window_handles
browser.close()

In [23]:
driver = webdriver.Chrome(path)
driver.get('https://www.naver.com')

In [24]:
driver.find_element_by_css_selector('.an_a.mn_mail').click()

In [25]:
%%writefile account.json
{
    "id":"아이디",
    "pw":"패스워드"
}

Writing account.json


In [35]:
import json
with open('account.json','r',encoding='UTF-8') as f: 
    account = json.load(f)

## id

In [36]:
driver.find_element_by_id('id').clear()
driver.find_element_by_id('pw').clear()
driver.find_element_by_id('id').send_keys(account['id'])
driver.find_element_by_id('pw').send_keys(account['pw'])

## name

In [37]:
driver.find_element_by_name('id').clear()
driver.find_element_by_name('pw').clear()
driver.find_element_by_name('id').send_keys(account['id'])
driver.find_element_by_name('pw').send_keys(account['pw'])

## css

In [38]:
driver.find_element_by_css_selector('#id').clear()
driver.find_element_by_css_selector('#pw').clear()
driver.find_element_by_css_selector('#id').send_keys(account['id'])
driver.find_element_by_css_selector('#pw').send_keys(account['pw'])

## xpath

In [39]:
driver.find_element_by_xpath('//input[@name="id"]').clear()
driver.find_element_by_xpath('//input[@name="pw"]').clear()
driver.find_element_by_xpath('//input[@name="id"]').send_keys(account['id'])
driver.find_element_by_xpath('//input[@name="pw"]').send_keys(account['pw'])

In [40]:
driver.find_element_by_class_name('btn_global').click()

## 받은메일함

In [41]:
driver.get('https://mail.naver.com/')

In [42]:
for tag in driver.find_elements_by_css_selector('strong.mail_title'):
    print(tag.text)


## 스팸메일함

In [46]:
driver.find_element_by_css_selector('span.item_wrap.bu6').click()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"span.item_wrap.bu6"}
  (Session info: chrome=77.0.3865.90)


In [44]:
for tag in driver.find_elements_by_css_selector('strong.mail_title'):
    print(tag.text)

## Cookies

In [47]:
driver.get_cookies()

[{'domain': '.nid.naver.com',
  'expiry': 1884753127,
  'httpOnly': False,
  'name': 'nid_buk',
  'path': '/',
  'secure': True,
  'value': '4YI6CKXJBGFV2'},
 {'domain': '.nid.naver.com',
  'expiry': 1577169774.868836,
  'httpOnly': False,
  'name': 'nid_enctp',
  'path': '/',
  'secure': False,
  'value': '1'},
 {'domain': '.nid.naver.com',
  'expiry': 1600929785,
  'httpOnly': False,
  'name': 'nid_slevel',
  'path': '/',
  'secure': False,
  'value': '1'},
 {'domain': '.naver.com',
  'expiry': 2524640398.409768,
  'httpOnly': False,
  'name': 'NNB',
  'path': '/',
  'secure': False,
  'value': '4YI6CKXJBGFV2'}]

In [48]:
# driver.get('https://mail.naver.com')
# driver.page_source

In [49]:
# html = getDownload('https://mail.naver.com')
# html.text

In [50]:
from requests import Session

session = Session()

In [51]:
for cookie in driver.get_cookies():
    print(cookie)
    print()

{'domain': '.nid.naver.com', 'expiry': 1884753127, 'httpOnly': False, 'name': 'nid_buk', 'path': '/', 'secure': True, 'value': '4YI6CKXJBGFV2'}

{'domain': '.nid.naver.com', 'expiry': 1577169774.868836, 'httpOnly': False, 'name': 'nid_enctp', 'path': '/', 'secure': False, 'value': '1'}

{'domain': '.nid.naver.com', 'expiry': 1600929785, 'httpOnly': False, 'name': 'nid_slevel', 'path': '/', 'secure': False, 'value': '1'}

{'domain': '.naver.com', 'expiry': 2524640398.409768, 'httpOnly': False, 'name': 'NNB', 'path': '/', 'secure': False, 'value': '4YI6CKXJBGFV2'}



In [52]:
for cookie in driver.get_cookies():
    print(cookie['name'], cookie['value'])
    session.cookies.set(cookie['name'], cookie['value'])

nid_buk 4YI6CKXJBGFV2
nid_enctp 1
nid_slevel 1
NNB 4YI6CKXJBGFV2


## 새창

In [53]:
url = 'https://www.kt.com/'
driver.get(url)

In [54]:
driver.find_element_by_css_selector('.quick > a:first-child').get_attribute('outerHTML')

'<a href="javascript:floating.login();" title="새창 열림">로그인</a>'

In [55]:
driver.find_element_by_css_selector('.quick > a:first-child').click()

In [56]:
# driver.find_element_by_id('userId') => 새창에서 작업하고 있지 않기 때문에 error
driver.window_handles

['CDwindow-B66420AF085E51FA233346F14FAA3F3E',
 'CDwindow-42806BA0E72A2EF432E6EA1F3F03FA65']

In [57]:
driver.switch_to.window(driver.window_handles[1])

In [58]:
driver.find_element_by_id('userId').send_keys('test')